In [14]:
#| default_exp popgen

In [15]:
#| export
# chewc/popgen.py
from typing import Tuple
import jax
import jax.numpy as jnp
from numpy.random import default_rng


import msprime
# Correctly import the Population structure from your new module
from chewc.structs import Population, SimParam
from chewc.config import SimConfig

In [16]:
#|export
def quick_haplo(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_chr: int,
    n_loci_per_chr: int,
    max_pop_size: int,
    ploidy: int = 2,
    inbred: bool = False,
    chr_len_cm: float = 100.0
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new, padded founder population with random haplotypes.

    This function is designed to be run once at the beginning of a simulation
    on the host to generate the initial state. It produces arrays padded to
    `max_pop_size` to ensure fixed shapes for JIT compilation.

    Args:
        key: A JAX random key.
        n_ind: Number of founder individuals to create.
        n_chr: Number of chromosomes.
        n_loci_per_chr: Number of loci on each chromosome.
        max_pop_size: The total size of the arrays to pre-allocate.
        ploidy: The ploidy level of the individuals.
        inbred: If True, creates fully inbred individuals (homozygous at all loci).
        chr_len_cm: The length of each chromosome in centiMorgans for the genetic map.

    Returns:
        A tuple of (Population, genetic_map). The Population object will
        contain `n_ind` active individuals within arrays of size `max_pop_size`.
    """
    if n_ind > max_pop_size:
        raise ValueError("n_ind cannot be greater than max_pop_size.")

    key, geno_key, sex_key = jax.random.split(key, 3)

    # 1. Generate data for the initial `n_ind` founders
    if inbred:
        # For inbred lines, create one haplotype and tile it `ploidy` times.
        base_haplotypes = jax.random.randint(geno_key, (n_ind, n_chr, 1, n_loci_per_chr), 0, 2, dtype=jnp.uint8)
        founder_geno = jnp.tile(base_haplotypes, (1, 1, ploidy, 1))
    else:
        # For outbred founders, generate random alleles for all haplotypes.
        founder_geno = jax.random.randint(geno_key, (n_ind, n_chr, ploidy, n_loci_per_chr), 0, 2, dtype=jnp.uint8)

    founder_ids = jnp.arange(n_ind, dtype=jnp.int32)
    founder_sex = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))

    # Each founder allele gets a unique ID for IBD tracking.
    n_founder_alleles = n_ind * n_chr * ploidy * n_loci_per_chr
    founder_ibd_flat = jnp.arange(n_founder_alleles, dtype=jnp.uint32)
    founder_ibd = founder_ibd_flat.reshape(n_ind, n_chr, ploidy, n_loci_per_chr)
    if inbred:
        # Ensure IBD matches the inbred genotype structure.
        founder_ibd = jnp.tile(founder_ibd[:, :, 0:1, :], (1, 1, ploidy, 1))

    # 2. Pad all arrays to `max_pop_size` to ensure fixed shapes.
    # This is a core principle for JAX compatibility.
    n_pad = max_pop_size - n_ind

    padded_geno = jnp.pad(founder_geno, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=0)
    padded_ibd = jnp.pad(founder_ibd, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=-1)
    padded_id = jnp.pad(founder_ids, (0, n_pad), constant_values=-1)
    padded_sex = jnp.pad(founder_sex, (0, n_pad), constant_values=-1)

    # 3. The `is_active` mask is crucial for handling variable population sizes
    # within fixed-shape arrays.
    is_active = jnp.arange(max_pop_size) < n_ind

    # --- Generate a uniform genetic map ---
    loci_pos = jnp.linspace(0., chr_len_cm, n_loci_per_chr)
    genetic_map = jnp.tile(loci_pos, (n_chr, 1))

    # 4. Instantiate the Population dataclass with conforming shapes and dtypes.
    population = Population(
        geno=padded_geno,
        ibd=padded_ibd,
        id=padded_id,
        iid=jnp.arange(max_pop_size, dtype=jnp.int32),  # Internal index, always contiguous.
        mother=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        father=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        sex=padded_sex,
        gen=jnp.where(is_active, 0, -1).astype(jnp.int32), # Active individuals are gen 0
        is_active=is_active,
        # Initialize trait arrays with correct shape, even if no traits are defined yet.
        pheno=jnp.full((max_pop_size, 0), jnp.nan, dtype=jnp.float32),
        fixEff=jnp.zeros(max_pop_size, dtype=jnp.float32),
        bv=jnp.full((max_pop_size, 0), jnp.nan, dtype=jnp.float32),
    )

    return population, genetic_map

In [18]:
#| hide
import nbdev; nbdev.nbdev_export()